# Preparing the RASC Double Stars catalog for import into PiFinder

## Setup

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## importing the supplement

In [ ]:
df = pd.read_excel (r'./RASC DS supplement 210710-import.xlsx')
df.columns = df.columns.str.strip()
df.drop(['Unnamed: 10', 'Unnamed: 11'], axis=1, inplace=True)
# drop repeated headers
rows_to_drop = df[df.apply(lambda x: (x == df.columns).all(), axis=1)].index
# Drop the identified rows
df.drop(index=rows_to_drop, inplace=True)
print(df.describe())
df.head()

In [ ]:
df['season'] = None

# Variable to hold the current season value
current_season = None
seasons = {
    "WINTER (part 1)": "winter(1)",
    "WINTER (part 2)": "winter(2)",
    "SPRING (part 1)": "spring(1)",
    "SPRING (part 2)": "spring(2)", 
    "SPRING (part 3)": "spring(3)",
    "SUMMER (part 1)": "summer(1)",
    "SUMMER (part 2)": "summer(2)",
    "SUMMER (part 3)": "summer(2)",
    "AUTUMN (part 1)": "autumn(1)",
    "AUTUMN (part 2)": "autumn(2)",
}

# Iterate through the DataFrame
for index, row in df.iterrows():
    # Check if the 'Type' column is not NaN
    if pd.isna(row['Type']) and not pd.isna(row['Target']):
        # Update the current season using the value in the 'Target' column
        current_season = row['Target']
        #print(f"Updating to {current_season=} because of {row=}")
    else:
        # Set the current season for the row
        #print(current_season)
        df.at[index, 'season'] = seasons[current_season] if current_season is not None else nan
# Remove rows where 'Type' is NaN
df = df.dropna(subset=['Type'])
df['Pair'].fillna('AB', inplace=True)
df = df.loc[df['Target'] != 'Target']

In [ ]:
supplement_df = df
supplement_df


## importing the main catalog

In [ ]:
df = pd.read_excel (r'./RASC DS main checklist 210710.xlsx', skiprows=16)
df.columns = df.columns.str.strip()
# fix mismatch in name
df['Target'] = df['Target'].replace('The Trapezium', 'Trapezium')
df['Target'] = df['Target'].replace('U  Cyg', 'U Cyg')
df.drop(['Seen?'], axis=1, inplace=True)
# drop repeated headers
rows_to_drop = df[df.apply(lambda x: (x == df.columns).all(), axis=1)].index
# Drop the identified rows
df.drop(index=rows_to_drop, inplace=True)
df


In [ ]:
df.describe()

In [ ]:
# Remove rows where 'Type' is NaN
df = df.dropna(subset=['PSA'])

In [ ]:
main_df = df
main_df

## Consolidate

In [ ]:
print(main_df.columns)
print(supplement_df.columns)

In [ ]:
# Perform an outer join with supplement_df as the left DataFrame
merged_df = pd.merge(supplement_df, main_df, on='Target', how='left', suffixes=('_supp', ''))

# Strip all strings
merged_df = merged_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Print the columns of the merged DataFrame to verify the merge
print(merged_df.columns)

In [ ]:
merged_df.drop(['Con_supp', 'WDS'], axis=1, inplace=True)
# Keep rows where 'Target' column values are not 'Target'
merged_df = merged_df.loc[merged_df['Target'] != 'Target']
merged_df.rename(columns={
    'WDS_supp': 'WDS',
    'Sep': 'SepSec'
}, inplace=True)
merged_df = merged_df[['Target', 'Alternate ID', 'SAO', 'HIP', 'WDS', 'Con', 'RA 2000',
       'Dec 2000', 'Mm', 'X', 'PSA', 'season', 'Parent', 'Pair',
       'PA', 'SepSec', 'MagC', 'M1', 'M2', 'Notes', 'Type']]
merged_df = merged_df.fillna('')

In [ ]:
merged_df.describe()

## format for PiFinder

### first an export of the full dataset, before pruning

In [ ]:
merged_df.to_csv('rasc_double_stars_full.csv', sep='\t', index=False)

In [ ]:
df = merged_df.drop(['SAO', 'HIP', 'PSA', 'Parent'], axis=1, inplace=False)

### Group clusters

In [ ]:
non_empty_target = df['Target'] != ''

# Use cumsum on the boolean series to create group identifiers
df['group'] = non_empty_target.cumsum()
# Group the DataFrame by the 'group' column
grouped = df.groupby('group')

### Iterate over groups and extract consolidated notes

In [ ]:
# Create a new DataFrame
out_df = pd.DataFrame(columns=['Target', 'AlternateID', 'WDS', 'Con', 'RA2000', 'Dec2000', 'Mag', 'MaxSepSec', 'Notes'])

def extract_info(row):
    result = f"""{row['Pair']} ({row['Type']}), PA: {row['PA']}, Sep: {row['SepSec']}<SECS>, Mag: {row['M1']}/{row['M2']} {row['Notes']}"""
    return result.replace('"', '<SECS>')
    
# Iterate over each group
for group_number, group_df in grouped:
    #print(f"Processing group {group_number}, {len(group_df)}")
    # 'group_df' is a DataFrame containing only the rows from the current group

    new_row = {'Notes': ''}
    max_sep = -1
    # Perform your operations on each group member here
    # For example, let's print the first row of each group
    letterset = set()
    for index, row in group_df.iterrows():
        pairs = row['Pair'].split(',')
        for pair in pairs:
            letters = [*pair]
            if letters[1].islower():
                letterset.discard(letters[0])
                letters = [letters[1]]
            letterset.update(letters)
    #print(f"{group_number=}, {letterset=}")
    if len(letterset) != len(group_df)+1:
        print(f"Lengnth of letterset is {len(letterset)} == {len(group_df)+1}, {letterset=}, {group_number=}, {group_df=}")
    nr_stars = len(letterset)
         
    for index, row in group_df.iterrows():
        if row['SepSec'] > max_sep:
            max_sep = row['SepSec']
        if row['Target']:
            new_row = {
            'Target': row['Target'],
            'AlternateID': row['Alternate ID'],
            'WDS': row['WDS'],
            'Con': row['Con'],
            'RA2000': row['RA 2000'], 
            'Dec2000': row['Dec 2000'],
            'Mag': [row['MagC'], row['M1'], row['M2']],
            'Notes': f"{nr_stars},󰥮{row['Mm']/10:g}cm,{row['X']:g}x,{row['season']}<NEWLINE>{extract_info(row)}"
            }
        else:
            new_row['Notes'] = new_row['Notes'] + '<NEWLINE>' + extract_info(row)

    # Append the new row to the DataFrame
    new_row['MaxSepSec'] = max_sep
    out_df = pd.concat([out_df,  pd.DataFrame([new_row])], ignore_index=True)
out_df

In [ ]:
out_df.index = out_df.index + 1
out_df = out_df.reset_index().rename(columns={'index': 'sequence'})
out_df.to_csv('rasc_double_stars.csv', sep='\t', index=False)